# Proyek Analisis Data: E-Commerce Public Dataset
- **Nama:** Faaris Muda Dwi Nugraha
- **Email:** faarismudadwinugraha@gmail.com / faarismudawork@gmail.com
- **ID Dicoding:** faarismuda

## Menentukan Pertanyaan Bisnis

**1. "Bagaimana performa rata-rata waktu pengiriman di tiap kota pada tahun 2017, dan apakah ada kota dengan waktu pengiriman lebih dari 10 hari yang dapat memperburuk kepuasan pelanggan?"**

- **Specific**: Fokus pada analisis waktu pengiriman di tiap kota pada tahun 2017.
- **Measurable**: Mengukur rata-rata waktu pengiriman dan identifikasi kota dengan waktu pengiriman lebih dari 10 hari.
- **Action-oriented**: Dapat digunakan untuk mengidentifikasi area yang memerlukan perbaikan dalam hal logistik dan pengiriman.
- **Relevant**: Mengoptimalkan waktu pengiriman untuk meningkatkan pengalaman pelanggan dan kepuasan.
- **Time-bound**: Membatasi analisis pada tahun 2017.

**Teknik Analisis Lanjutan**:
- **Geospatial Analysis**: Menganalisis lokasi dan waktu pengiriman menggunakan data dari `orders_dataset.csv` dan `geolocation_dataset.csv`.
    - Membuat **heatmap** untuk menunjukkan waktu pengiriman di setiap kota.
    - Menghitung **rata-rata waktu pengiriman** per kota dan mengidentifikasi **outlier**.


**2. "Apa hubungan antara frekuensi pembelian pelanggan dan total pengeluaran mereka di tahun 2017, dan bagaimana pola ini memengaruhi segmentasi pelanggan yang paling menguntungkan?"**

- **Specific**: Fokus pada analisis frekuensi pembelian dan total pengeluaran pelanggan di tahun 2017.
- **Measurable**: Menghitung frekuensi pembelian dan total pengeluaran pelanggan.
- **Action-oriented**: Membantu dalam membuat strategi pemasaran yang ditujukan untuk segmen pelanggan yang menguntungkan.
- **Relevant**: Penting untuk strategi retensi pelanggan dan peningkatan pendapatan.
- **Time-bound**: Fokus pada data tahun 2017.

**Teknik Analisis Lanjutan**:
- **RFM Analysis**: Mengukur **recency**, **frequency**, dan **monetary** untuk memahami segmentasi pelanggan berdasarkan perilaku pembelian.
    - Menganalisis dataset untuk menghitung **frekuensi** dan **total pengeluaran**.
    - Segmentasi pelanggan berdasarkan frekuensi pembelian dan total pengeluaran untuk mengidentifikasi kelompok yang paling menguntungkan.


**3. "Produk kategori mana yang memberikan kontribusi terbesar terhadap pendapatan perusahaan selama Q1 2017, dan apakah ada kategori dengan tingkat pengembalian produk (return rate) yang lebih tinggi?"**

- **Specific**: Fokus pada kontribusi pendapatan berdasarkan kategori produk selama Q1 2017.
- **Measurable**: Menghitung kontribusi pendapatan dari tiap kategori produk dan mengukur tingkat pengembalian produk.
- **Action-oriented**: Menyediakan informasi untuk perbaikan manajemen inventaris dan pengelolaan kategori produk.
- **Relevant**: Membantu dalam memprioritaskan kategori produk untuk strategi pemasaran dan pengelolaan produk.
- **Time-bound**: Fokus pada periode Q1 2017.

**Teknik Analisis Lanjutan**:
- **Clustering**: Mengelompokkan produk berdasarkan kategori dan kontribusi pendapatan untuk melihat pola kategori yang lebih menguntungkan.
    - Menganalisis data dataset untuk menghitung kontribusi pendapatan berdasarkan kategori produk.
    - Mengukur **tingkat pengembalian produk** dan mengidentifikasi kategori dengan **return rate** tinggi.

## Import Semua Packages/Library yang Digunakan

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

## Data Wrangling

### Gathering Data

In [ ]:
# Baca data dari file CSV
customers = pd.read_csv("https://raw.githubusercontent.com/faarismuda/E-commerce-Analysis/main/Dataset/customers_dataset.csv")
geolocation = pd.read_csv("https://raw.githubusercontent.com/faarismuda/E-commerce-Analysis/main/Dataset/geolocation_dataset.csv")
order_items = pd.read_csv("https://raw.githubusercontent.com/faarismuda/E-commerce-Analysis/main/Dataset/order_items_dataset.csv")
order_payments = pd.read_csv("https://raw.githubusercontent.com/faarismuda/E-commerce-Analysis/main/Dataset/order_payments_dataset.csv")
order_reviews = pd.read_csv("https://raw.githubusercontent.com/faarismuda/E-commerce-Analysis/main/Dataset/order_reviews_dataset.csv")
orders = pd.read_csv("https://raw.githubusercontent.com/faarismuda/E-commerce-Analysis/main/Dataset/orders_dataset.csv", parse_dates=['order_purchase_timestamp', 'order_delivered_customer_date'])
product_category_name_translation = pd.read_csv("https://raw.githubusercontent.com/faarismuda/E-commerce-Analysis/main/Dataset/product_category_name_translation.csv")
products = pd.read_csv("https://raw.githubusercontent.com/faarismuda/E-commerce-Analysis/main/Dataset/products_dataset.csv")
sellers = pd.read_csv("https://raw.githubusercontent.com/faarismuda/E-commerce-Analysis/main/Dataset/sellers_dataset.csv")

**Q1**

In [ ]:
# Filter orders dari tahun 2017
orders['order_year'] = orders['order_purchase_timestamp'].dt.year
orders_2017 = orders[orders['order_year'] == 2017]

# Hitung waktu pengiriman
orders_2017['delivery_duration'] = (orders_2017['order_delivered_customer_date'] - orders_2017['order_purchase_timestamp']).dt.days

# Gabungkan orders dengan customers untuk mendapatkan informasi kota
orders_customers = pd.merge(orders_2017, customers, on='customer_id', how='left')

# Gabung dengan reviews untuk mendapatkan skor review
orders_customers_reviews = pd.merge(orders_customers, reviews[['order_id', 'review_score']], on='order_id', how='left')

# Gabung dengan geolocation untuk mendapatkan latitude dan longitude
geolocation_unique = geolocation.drop_duplicates(subset=['geolocation_zip_code_prefix', 'geolocation_city'])
merged_data = pd.merge(orders_customers_reviews, geolocation_unique,
                       left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix',
                       how='left')

# Final dataset
data_q1 = merged_data[['order_id', 'customer_city', 'delivery_duration', 'review_score',
                          'geolocation_lat', 'geolocation_lng']]

# Simpan dataset ke CSV
data_q1.to_csv('data_q1.csv', index=False)

data_q1.head()

**Q2**

In [ ]:
# Filter orders dari tahun 2017
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders_2017 = orders[orders['order_purchase_timestamp'].dt.year == 2017]

# Gabungkan orders_2017 dengan order_items
data_q2 = orders_2017.merge(order_items, on='order_id', how='inner')

# Gabungkan dengan customers untuk mendapatkan customer_unique_id
data_q2 = data_q2.merge(customers, on='customer_id', how='inner')

# Gabungkan dengan order_payments untuk validasi total pembayaran
data_q2 = data_q2.merge(order_payments, on='order_id', how='left')

# Tambahkan kolom total pengeluaran (price + freight_value)
data_q2['total_spent'] = data_q2['price'] + data_q2['freight_value']

# Final dataset
data_q2 = data_q2[['customer_unique_id', 'order_id', 'order_purchase_timestamp',
                               'price', 'freight_value', 'total_spent',
                               'payment_value', 'payment_type']]

# Simpan dataset ke CSV
data_q2.to_csv('combined_dataset_2017.csv', index=False)

data_q2.head()

**Q3**

**Insight:**
- xxx
- xxx

### Assessing Data

**Q1**

In [ ]:
# 1. Melihat informasi umum
print("Informasi Dataset:")
print(data_q1.info())

# 2. Statistik Deskriptif
print("\nStatistik Deskriptif:")
print(data_q1.describe(include='all'))

# 3. Memeriksa nilai yang hilang
print("\nJumlah Nilai yang Hilang per Kolom:")
print(data_q1.isnull().sum())

# 4. Memeriksa duplikasi
print("\nJumlah Baris Duplikat:")
print(data_q1.duplicated().sum())

# 5. Melihat nilai unik dalam kolom kategorikal
print("\nNilai Unik Kolom 'customer_city':")
print(data_q1['customer_city'].unique())

**Q2**

In [ ]:
# 1. Melihat informasi umum
print("Informasi Dataset:")
print(data_q2.info())

# 2. Statistik Deskriptif
print("\nStatistik Deskriptif:")
print(data_q2.describe(include='all'))

# 3. Memeriksa nilai yang hilang
print("\nJumlah Nilai yang Hilang per Kolom:")
print(data_q2.isnull().sum())

# 4. Memeriksa duplikasi
print("\nJumlah Baris Duplikat:")
print(data_q2.duplicated().sum())

# 5. Melihat nilai unik
print("\nCek Nilai Unik:")
print(data_q2.nunique())

**Q3**

**Insight:**
- xxx
- xxx

### Cleaning Data

**Q1**

In [ ]:
# 1. Menghapus duplikasi
data_q1_cleaned = data_q1.drop_duplicates()
print("\nSetelah Menghapus Duplikasi, Total Baris:", len(data_q1_cleaned))

# 2. Menghapus baris dengan nilai yang hilang secara signifikan (jika diperlukan)
# Misalnya, kita menghapus baris dengan `delivery_duration` atau `review_score` kosong
data_q1_cleaned = data_q1_cleaned.dropna(subset=['delivery_duration', 'review_score'])

# 3. Mengisi nilai yang hilang (jika sesuai)
# Misalnya, untuk kolom koordinat geografis
data_q1_cleaned['geolocation_lat'].fillna(data_q1_cleaned['geolocation_lat'].mean(), inplace=True)
data_q1_cleaned['geolocation_lng'].fillna(data_q1_cleaned['geolocation_lng'].mean(), inplace=True)

# 4. Memastikan tipe data sesuai
# Kolom seperti `delivery_duration` dan `review_score` harus bertipe numerik
data_q1_cleaned['delivery_duration'] = data_q1_cleaned['delivery_duration'].astype(float)
data_q1_cleaned['review_score'] = data_q1_cleaned['review_score'].astype(float)

# 5. Validasi nilai aneh atau outlier
# Contoh: Durasi pengiriman negatif tidak masuk akal
outliers = data_q1_cleaned[data_q1_cleaned['delivery_duration'] < 0]
print("\nBaris dengan Durasi Pengiriman Negatif (Outlier):")
print(outliers)

# Menghapus outlier
data_q1_cleaned = data_q1_cleaned[data_q1_cleaned['delivery_duration'] >= 0]

# Simpan dataset yang sudah bersih
data_q1_cleaned.to_csv('data_q1_cleaned.csv', index=False)

print("\nDataset telah dibersihkan. Total Baris Setelah Cleaning:", len(data_q1_cleaned))


**Q2**

In [ ]:
# 2. Data Cleaning
# a. Menghapus duplikasi (jika ada)
data_q2_cleaned = data_q2.drop_duplicates()

# b. Mengatasi missing values
# Untuk kolom yang penting seperti `total_spent` atau `order_purchase_timestamp`, missing values akan dianalisis lebih lanjut
print("\nMissing values setelah pembersihan duplikasi:")
print(data_q2_cleaned.isnull().sum())

# Jika ada missing values di kolom 'total_spent', kita bisa menghapus baris tersebut atau mengisi dengan mean/median (opsional)
if data_q2_cleaned['total_spent'].isnull().sum() > 0:
    data_q2_cleaned['total_spent'].fillna(data_q2_cleaned['total_spent'].median(), inplace=True)  # Contoh mengisi median

# c. Konversi tipe data jika diperlukan
data_q2_cleaned['order_purchase_timestamp'] = pd.to_datetime(data_q2_cleaned['order_purchase_timestamp'])  # Pastikan kolom datetime benar

# d. Validasi konsistensi data: memastikan `total_spent` >= 0
invalid_total_spent = data_q2_cleaned[data_q2_cleaned['total_spent'] < 0]
if not invalid_total_spent.empty:
    print("\nBaris dengan 'total_spent' tidak valid (kurang dari 0):")
    print(invalid_total_spent)
    # Menghapus baris dengan total_spent negatif
    data_q2_cleaned = data_q2_cleaned[data_q2_cleaned['total_spent'] >= 0]

# e. Validasi `payment_value` vs `total_spent`
# Jika ada selisih yang terlalu besar, data bisa dianalisis lebih lanjut atau dihapus
data_q2_cleaned['payment_difference'] = abs(data_q2_cleaned['payment_value'] - data_q2_cleaned['total_spent'])
invalid_payments = data_q2_cleaned[data_q2_cleaned['payment_difference'] > 5]  # Toleransi selisih 5
if not invalid_payments.empty:
    print("\nBaris dengan perbedaan signifikan antara 'payment_value' dan 'total_spent':")
    print(invalid_payments)

# Opsional: Drop baris dengan payment_difference besar (jika dianggap noise)
data_q2_cleaned = data_q2_cleaned[data_q2_cleaned['payment_difference'] <= 5]

# f. Menambahkan kolom baru untuk analisis
# Kolom 'month' untuk analisis musiman
data_q2_cleaned['month'] = data_q2_cleaned['order_purchase_timestamp'].dt.month

# Melihat hasil data setelah cleaning
print("\nData Setelah Cleaning:")
print(data_q2_cleaned.head())

# Simpan hasil clean dataset (opsional)
data_q2_cleaned.to_csv('data_q2_cleaned.csv', index=False)


**Q3**

**Insight:**
- xxx
- xxx

## Exploratory Data Analysis (EDA)

### Explore ...

**Insight:**
- xxx
- xxx

## Visualization & Explanatory Analysis

### Pertanyaan 1:

**Insight:**
- xxx
- xxx

### Pertanyaan 2:

**Insight:**
- xxx
- xxx

### Pertanyaan 3:

**Insight:**
- xxx
- xxx

## Analisis Lanjutan

## Conclusion

- Conclusion pertanyaan 1
- Conclusion pertanyaan 2
- Conclusion pertanyaan 3